In [ ]:
# Import Libraries


In [ ]:
# Import Michele's VHR Snow Depth Data (Apr 9, 2022)
# Input File

VHR_sd = pd.ExcelFile("/Users/Adrian/OneDrive - University of New Mexico/SNOW RESEARCH/SNOW DATA/Banded Peak Ranch/SNOTEL DATA/BPR_SNOTELs_2016-2023.xlsx")

In [ ]:
# Plot VHR Snow Depth Data (before resampling)


In [ ]:
# Resample VHR Snow Depth: 3m to 90m

import rasterio
from rasterio.enums import Resampling

def resample_geotiff(input_path, output_path, scale_factor):
    with rasterio.open(input_path) as src:
        # Read the original resolution and calculate the new resolution
        orig_resolution = src.res[0]
        new_resolution = orig_resolution * scale_factor

        # Calculate the new width and height based on the scale factor
        new_width = int(src.width * (orig_resolution / new_resolution))
        new_height = int(src.height * (orig_resolution / new_resolution))

        # Resample the GeoTIFF
        data = src.read(
            out_shape=(src.count, new_height, new_width),
            resampling=Resampling.bilinear
        )

        # Update the metadata with the new resolution
        transform = src.transform * src.transform.scale(
            (src.width / new_width),
            (src.height / new_height)
        )

        # Write the resampled data to a new GeoTIFF file
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=new_height,
            width=new_width,
            count=src.count,
            dtype=data.dtype,
            crs=src.crs,
            transform=transform,
        ) as dst:
            dst.write(data)

if __name__ == "__main__":
    # Replace 'input.tif' and 'output.tif' with your file paths
    input_file = 'input.tif'
    output_file = 'output.tif'
    
    # Set the scale factor (e.g., from 3m to 90m)
    scale_factor = 90.0 / 3.0
    
    # Resample the GeoTIFF
    resample_geotiff(input_file, output_file, scale_factor)


In [ ]:
# Resample and Clip VHR Snow Depth to Catchment

import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.enums import Resampling

def resample_geotiff(input_path, output_path, scale_factor):
    with rasterio.open(input_path) as src:
        orig_resolution = src.res[0]
        new_resolution = orig_resolution * scale_factor
        new_width = int(src.width * (orig_resolution / new_resolution))
        new_height = int(src.height * (orig_resolution / new_resolution))

        data = src.read(
            out_shape=(src.count, new_height, new_width),
            resampling=Resampling.bilinear
        )

        transform = src.transform * src.transform.scale(
            (src.width / new_width),
            (src.height / new_height)
        )

        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=new_height,
            width=new_width,
            count=src.count,
            dtype=data.dtype,
            crs=src.crs,
            transform=transform,
        ) as dst:
            dst.write(data)

def clip_geotiff_to_shapefile(input_geotiff, output_geotiff, shapefile):
    with rasterio.open(input_geotiff) as src:
        with gpd.read_file(shapefile) as shape:
            # Make sure the coordinate systems match
            shape = shape.to_crs(src.crs)

            # Clip the raster to the shapefile geometry
            clipped_data, clipped_transform = mask(src, shape.geometry, crop=True)

            # Update metadata with the new transform
            profile = src.profile
            profile.update({
                'height': clipped_data.shape[1],
                'width': clipped_data.shape[2],
                'transform': clipped_transform,
            })

            # Write the clipped data to a new GeoTIFF file
            with rasterio.open(output_geotiff, 'w', **profile) as dst:
                dst.write(clipped_data)

if __name__ == "__main__":
    # Replace these paths with your file paths
    input_file = 'resampled.tif'
    output_file = 'clipped_resampled.tif'
    shapefile_path = 'clip_shapefile.shp'

    # Set the scale factor (e.g., from 3m to 90m)
    scale_factor = 90.0 / 3.0
    
    # Resample the GeoTIFF
    resample_geotiff(input_file, output_file, scale_factor)

    # Clip the resampled GeoTIFF to the shapefile
    clip_geotiff_to_shapefile(output_file, 'final_clip.tif', shapefile_path)


In [ ]:
# Plot resampled and clipped VHR Snow Depth


In [ ]:
# Export as a netCDF?
